In [1]:
import ROOT as rt
from pprint import pprint
import numpy as np

Welcome to JupyROOT 6.20/04


In [98]:
def write_tree_info_to_txt(infile, outtxt, keep_2P2F=True, keep_3P1F=True):
    """
    Write info from TFile `infile` from TTree 'passedEvents' to `outtxt`.

    Info which gets written:
    Run : LumiSect : Event
    """
    tfile = rt.TFile.Open(infile)
    tree = tfile.Get("passedEvents")

    with open(outtxt, "w") as f:
        f.write("# Run : LumiSect : Event\n")
        for evt in tree:
            keep_evt = True if (keep_2P2F and evt.is2P2F) or (keep_3P1F and evt.is3P1F) else False
            if keep_evt:
                f.write(f"{evt.Run} : {evt.LumiSect} : {evt.Event}\n")

def get_list_of_lines(evt_ls_txt):
    """
    Return a list of lines starting with a digit and
    with a trailing newline ('\\n') stripped.
    """
    with open(evt_ls_txt, "r") as f:
        return [line.rstrip('\n') for line in f.readlines() if line[0].isdigit()]

def get_list_of_tuples(evt_ls):
    """
    Return a list of 3-tuples from a list of strings `evt_ls`:

    [
        (Run1, LumiSect1, Event1),
        (Run2, LumiSect2, Event2),
        ...
    ]
    """
    new_evt_ls = []
    for line in evt_ls:
        tup = tuple([int(num) for num in line.split(":")])
        new_evt_ls.append(tup)
    return new_evt_ls

class FileComparer:

    def __init__(self, txtpath_jake, txtpath_elis, control_reg=""):
        """
        Feed in two txt files to be compared.

        Each txt file is converted to a list of 3-tuples and stored.
        """
        self.cr = control_reg
        self.check_cr(txtpath_jake, txtpath_elis)
        if control_reg in "":
            self.cr = "all"

        self.ls_of_tup_jake = get_list_of_tuples(get_list_of_lines(txtpath_jake))
        self.ls_of_tup_elis = get_list_of_tuples(get_list_of_lines(txtpath_elis))

    def check_cr(self, path1, path2):
        """Make sure that the control region is the one requested."""
        cr_low = self.cr.lower()
        assert cr_low in ("2p2f", "3p1f", "")
        msg = f"The `control_reg` ({self.cr}) not found in names of txt files."
        assert all(cr_low in f.lower() for f in (path1, path2)), msg

    def compare_files(self):
        """
        """
        self.set_common_to_both = set(self.ls_of_tup_elis) & set(self.ls_of_tup_jake)
        # self.set_common_to_both = set(self.ls_of_tup_elis).intersection(set(self.ls_of_tup_jake))
        self.set_unique_to_elis = set(self.ls_of_tup_elis) - set(self.ls_of_tup_jake)
        self.set_unique_to_jake = set(self.ls_of_tup_jake) - set(self.ls_of_tup_elis)

    def print_results(self, whose="all", show_n_evts=25):
        """Print info describing differences between two files."""
        print(f"Comparing {self.cr.upper()}:")
        print(f"{'n_evts total Elisa:':<25}{len(self.ls_of_tup_elis)}")
        print(f"{'n_evts total Jake:':<25}{len(self.ls_of_tup_jake)}")
        print(f"{'n_evts in common:':<25}{len(self.set_common_to_both)}")
        print(f"{'n_evts unique to Elisa:':<25}{len(self.set_unique_to_elis)}")
        print(f"{'n_evts unique to Jake:':<25}{len(self.set_unique_to_jake)}")
        header = "#-- Run -- LumiSect -- Event --#"
        if show_n_evts == -1:
            show_n_evts = None
        if whose in ("jake", "all"):
            print(f"  Jake's unique events:")
            print(header)
            pprint(list(self.set_unique_to_jake)[:show_n_evts])
            print()
        if whose in ("elisa", "all"):
            print(f"  Elisa's unique events:")
            print(header)
            pprint(list(self.set_unique_to_elis)[:show_n_evts])
            print()
        

In [100]:
infile_jake_tree = "/blue/avery/rosedj1/ZplusXpython/data/ZLL_CR_FRapplied/Data_2018_NoDuplicates_RunEventLumi.root"

infile_elisa       = "/blue/avery/rosedj1/ZplusXpython/data/elisa/CRLLos_listOfEvents.txt"
infile_elisa_2p2f  = "/blue/avery/rosedj1/ZplusXpython/data/elisa/CRLLos_2P2F_listOfEvents.txt"
infile_elisa_3p1f  = "/blue/avery/rosedj1/ZplusXpython/data/elisa/CRLLos_3P1F_listOfEvents.txt"

infile_jake      = "/blue/avery/rosedj1/ZplusXpython/data/elisa/CRLLos_listOfEvents_jake.txt"
infile_jake_2p2f = "/blue/avery/rosedj1/ZplusXpython/data/elisa/CRLLos_listOfEvents_jake_2P2F.txt"
infile_jake_3p1f = "/blue/avery/rosedj1/ZplusXpython/data/elisa/CRLLos_listOfEvents_jake_3P1F.txt"
# write_tree_info_to_txt(tree, infile_jake)

## Make txt files of events.

In [22]:
#--- 2P2F ---#
# write_tree_info_to_txt(infile_jake_tree, infile_jake_2p2f, keep_2P2F=True, keep_3P1F=False)

#--- 3P1F ---#
# write_tree_info_to_txt(infile_jake_tree, infile_jake_3p1f, keep_2P2F=False, keep_3P1F=True)

## Compare files.

In [103]:
fc_jakevselisa_3p1f = FileComparer(infile_jake_3p1f, infile_elisa_3p1f, control_reg="3p1f")
fc_jakevselisa_3p1f.compare_files()
fc_jakevselisa_3p1f.print_results(whose="jake", show_n_evts=10)

fc_jakevselisa_2p2f = FileComparer(infile_jake_2p2f, infile_elisa_2p2f, control_reg="2p2f")
fc_jakevselisa_2p2f.compare_files()
fc_jakevselisa_2p2f.print_results(whose="jake", show_n_evts=10)

fc_jakevselisa_all = FileComparer(infile_jake, infile_elisa, control_reg="")
fc_jakevselisa_all.compare_files()
fc_jakevselisa_all.print_results(whose="jake", show_n_evts=10)

Comparing 3P1F:
n_evts total Elisa:      4806
n_evts total Jake:       4150
n_evts in common:        4035
n_evts unique to Elisa:  770
n_evts unique to Jake:   26
  Jake's unique events:
#-- Run -- LumiSect -- Event --#
[(317182, 1329, 1905692245),
 (320038, 548, 845311759),
 (317640, 452, 638092741),
 (319524, 151, 172084101),
 (319524, 143, 160539721),
 (317591, 285, 372742783),
 (318874, 125, 203908118),
 (317661, 517, 748042526),
 (319639, 842, 1246930568),
 (318876, 34, 51539109)]

Comparing 2P2F:
n_evts total Elisa:      46067
n_evts total Jake:       43608
n_evts in common:        43014
n_evts unique to Elisa:  3052
n_evts unique to Jake:   505
  Jake's unique events:
#-- Run -- LumiSect -- Event --#
[(317696, 592, 881210461),
 (320065, 665, 1052355714),
 (317392, 448, 600937506),
 (317640, 771, 1135540995),
 (319579, 2984, 4520126795),
 (317320, 1233, 1832145181),
 (319756, 761, 1239333999),
 (317649, 181, 267164819),
 (319756, 804, 1309932845),
 (319656, 58, 5229496)]

Compari

In [109]:
print("JAKE")
print("all")
print(len(fc_jakevselisa_all.ls_of_tup_jake))
print(len(set(fc_jakevselisa_all.ls_of_tup_jake)))

print("2P2F")
print(len(fc_jakevselisa_2p2f.ls_of_tup_jake))
print(len(set(fc_jakevselisa_2p2f.ls_of_tup_jake)))

print("3P1F")
print(len(fc_jakevselisa_3p1f.ls_of_tup_jake))
print(len(set(fc_jakevselisa_3p1f.ls_of_tup_jake)))
print("#" + "-"*30 + "#")

print("ELISA")
print("all")
print(len(fc_jakevselisa_all.ls_of_tup_elis))
print(len(set(fc_jakevselisa_all.ls_of_tup_elis)))

print("2P2F")
print(len(fc_jakevselisa_2p2f.ls_of_tup_elis))
print(len(set(fc_jakevselisa_2p2f.ls_of_tup_elis)))

print("3P1F")
print(len(fc_jakevselisa_3p1f.ls_of_tup_elis))
print(len(set(fc_jakevselisa_3p1f.ls_of_tup_elis)))

JAKE
all
47758
47580
2P2F
43608
43519
3P1F
4150
4061
#------------------------------#
ELISA
all
50873
50753
2P2F
46067
46066
3P1F
4806
4805
